### ПАКЕТ SURPRISE

* используйте данные MovieLens 1M
* использовать любые модели из пакета
* получите RMSE на тестовом сете 0.87 и ниже

#### Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [2]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [3]:
movies = pd.read_csv('ml_latest/movies.csv')
ratings = pd.read_csv('ml_latest/ratings.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


In [7]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [8]:
movies_with_ratings

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,4.0,1.113766e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,5.0,9.488858e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.5,1.442169e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,4.0,1.370810e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,22.0,4.0,1.237623e+09
...,...,...,...,...,...,...
27757648,193878,Les tribulations d'une caissière (2011),Comedy,176871.0,2.0,1.537875e+09
27757649,193880,Her Name Was Mumu (2016),Drama,81710.0,2.0,1.537886e+09
27757650,193882,Flora (2017),Adventure|Drama|Horror|Sci-Fi,33330.0,2.0,1.537891e+09
27757651,193886,Leal (2018),Action|Crime|Drama,206009.0,2.5,1.537918e+09


In [9]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [1]:
dataset.head()

NameError: name 'dataset' is not defined

In [11]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [12]:
trainset, testset = train_test_split(data, test_size=.50)

In [11]:
movies_with_ratings.userId.value_counts()

123100.0    23715
117490.0     9279
134596.0     8381
212343.0     7884
242683.0     7515
            ...  
223518.0        1
10564.0         1
148678.0        1
148663.0        1
185369.0        1
Name: userId, Length: 283228, dtype: int64

In [12]:
movies_with_ratings.title.value_counts()

Shawshank Redemption, The (1994)               97999
Forrest Gump (1994)                            97040
Pulp Fiction (1994)                            92406
Silence of the Lambs, The (1991)               87899
Matrix, The (1999)                             84545
                                               ...  
Khubsoorat (1980)                                  1
Echoes From a Sombre Empire (1990)                 1
Mahanati (2018)                                    1
Camarón: When Flamenco Became Legend (2005)        1
Maliglutit (Searchers) (2016)                      1
Name: title, Length: 53817, dtype: int64

In [ ]:
# юзеров в 4 раза больше, чем фильмов
# поэтому будем использовать подход user-based
algo = KNNBasic(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...


In [26]:
import numpy as np
from sklearn.model_selection import KFold
X = dataset
kf = KFold(n_splits=5)
rm_0 = 0.87

models = []

for train_index, test_index in kf.split(X):
    
    algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
#    trainset, testset = X.iloc[train_index], X.iloc[test_index]
    data_train = Dataset.load_from_df(trainset, reader)
    algo.fit(data_train)
    
    data_test = Dataset.load_from_df(testset, reader)
    test_pred = algo.test(data_test)
    rm = accuracy.rmse(test_pred, verbose=True)
    
    if rm < rm_0:
        models.append(algo)
        rm_0 = rm
rm_0

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

trainset, testset = train_test_split(data, test_size=.80)
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
#trainset, testset = X.iloc[train_index], X.iloc[test_index]

algo.fit(trainset)
    
#data_test = Dataset.load_from_df(testset, reader)
test_pred = algo.test(testset)
rm = accuracy.rmse(test_pred, verbose=True)
rm < 0.87

Estimating biases using als...
